In [1]:
import cv2
import threading
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [2]:
import serial
import time
from queue import Queue
arduino = serial.Serial(port = '/dev/ttyUSB1', baudrate=9600, timeout=0.1)
data_queue = Queue()

In [3]:
joint_list = [[7,6,5], [11,10,9], [15,14,13], [18,18,17]]

In [4]:
def send_to_arduino():
    while True:
        if not data_queue.empty():
            data = data_queue.get()
            try:
                arduino.write(bytes(data, 'utf-8'))
                time.sleep(0.005)
            except Exception as e:
                print(f"Error: {e}")

In [5]:
def draw_finger_angles(image, results, joint_list):
    for hand in results.multi_hand_landmarks:
        angles = []
        for joint in joint_list:
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) # first coordinate
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) # second coordinate
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) # third coordinate

            radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
            angle = np.abs(radians*180.0/np.pi)
            if angle>180.0:
                angle = 360-angle
            angles.append(angle)
            cv2.putText(image, str(round(angle,2)), tuple(np.multiply(b, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        if (angles[0]<170) and (angles[1]<170) and (angles[2]<170):
            data_queue.put('0')
            cv2.putText(image, "Close", tuple(np.multiply(b, [600, 240]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 2, (240,16,91),5, cv2.LINE_AA)
        if (angles[0]>170) and (angles[1]>170) and (angles[2]>170):
            data_queue.put('1')
            cv2.putText(image, "Open", tuple(np.multiply(b, [600, 240]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 2, (240,16,91),5, cv2.LINE_AA)
    return image

In [6]:
def hand_detection():
    try:
        cap = cv2.VideoCapture(0)
        frame_counter = 0
        frame_skip = 2
        #coordinates = []
        with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
            while cap.isOpened():
                frame_counter +=1
                ret, frame = cap.read()
                if frame_counter % frame_skip !=0:
                    continue
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image = cv2.flip(image, 1)
                image.flags.writeable = False
                results = hands.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                if results.multi_hand_landmarks:
                    for num, hand in enumerate(results.multi_hand_landmarks):
                        mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                            )
                    #if get_label(num, hand, results):
                        #text, coord = get_label(num, hand, results)
                        #cv2.putText(image, text, coord, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    draw_finger_angles(image, results, joint_list)
                #show_coordinate(image, results, coordinates)
                cv2.imshow('Hand Tracking', image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [7]:
if __name__ == "__main__":
    hand_thread = threading.Thread(target=hand_detection, daemon=True)
    arduino_thread = threading.Thread(target=send_to_arduino, daemon=True)

    hand_thread.start()
    arduino_thread.start()

    hand_thread.join()
    arduino_thread.join()

I0000 00:00:1738324678.672949   27590 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1738324678.674583   27600 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.3), renderer: Mesa Intel(R) UHD Graphics (TGL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1738324678.696860   27593 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738324678.714360   27594 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ritushwar/.venv/lib/python3.12/site-packages/cv2/qt/plugins"
W0000 00:00:1738324680.806995   27594 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSI

KeyboardInterrupt: 

In [17]:
hand_detection()

[ WARN:5@203.462] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:5@203.463] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
I0000 00:00:1738323269.569642   23781 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1738323269.571317   24108 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.3), renderer: Mesa Intel(R) UHD Graphics (TGL GT2)
W0000 00:00:1738323269.603608   24102 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1738323269.620535   24103 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
